In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import pygad
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt


In [36]:
# Load CCPP dataset (make sure you have 'Folds5x2_pp.csv' locally or download from UCI)
data = pd.read_csv("Folds5x2_pp.csv")
X = data.drop(columns="PE").values
y = data["PE"].values

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [37]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MLP(nn.Module):
    def __init__(self, input_dim, n_hidden, neuron_list):
        super(MLP, self).__init__()
        layers = []
        in_features = input_dim
        for i in range(n_hidden):
            out_features = neuron_list[i]
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            in_features = out_features
        layers.append(nn.Linear(in_features, 1))  # Output layer
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)





In [38]:
def fitness_func(ga_instance, solution, solution_idx):
    import torch
    from torch import nn
    from torch.utils.data import DataLoader, TensorDataset, random_split

    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    print(device)
    n_hidden = int(round(solution[0]))
    neurons = [int(round(n)) for n in solution[1:6]]
    log_lr = solution[6]
    learning_rate = 10 ** log_lr

    try:
        model = MLP(X_train.shape[1], n_hidden, neurons).to(device)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        # Dataset
        X_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
        dataset = TensorDataset(X_tensor, y_tensor)

        # 80/20 split
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

        best_val_loss = float("inf")

        for epoch in range(20):  # match TensorFlow: epochs=20
            model.train()
            for xb, yb in train_loader:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                loss = criterion(model(xb), yb)
                loss.backward()
                optimizer.step()

            # Validation
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for xb, yb in val_loader:
                    xb, yb = xb.to(device), yb.to(device)
                    val_loss += criterion(model(xb), yb).item()
            val_loss /= len(val_loader)

            if val_loss < best_val_loss:
                best_val_loss = val_loss

        print(f"[{solution_idx}] layers={n_hidden}, neurons={neurons[:n_hidden]}, lr={learning_rate:.1e}, val_loss={best_val_loss:.4f}")
        return -best_val_loss

    except Exception as e:
        print(f"[{solution_idx}] FAILED: {e}")
        return -1e6


In [39]:
fitness_per_gen = []
def on_generation(ga_instance):
    fitness_per_gen.append(np.max(ga_instance.last_generation_fitness))
ga_instance = pygad.GA(
    num_generations=10,
    num_parents_mating=5,
    fitness_func=fitness_func,
    sol_per_pop=10,
    num_genes=7,
    on_generation=on_generation,
    gene_space=[
        {'low': 1, 'high': 5},        # n_hidden
        {'low': 10, 'high': 200},     # n1
        {'low': 10, 'high': 200},     # n2
        {'low': 10, 'high': 200},     # n3
        {'low': 10, 'high': 200},     # n4
        {'low': 10, 'high': 200},     # n5
        {'low': -4, 'high': -2}       # log10(lr)
    ],
    mutation_percent_genes=20,
    random_seed=42
)

ga_instance.run()
best_solution, _, _ = ga_instance.best_solution()
print("GA best solution:", best_solution)


mps
[0] layers=2, neurons=[191, 149], lr=1.3e-04, val_loss=7951.4125
mps
[1] layers=4, neurons=[124, 145, 14, 194], lr=2.7e-04, val_loss=68.2979
mps
[2] layers=2, neurons=[45, 68], lr=1.7e-03, val_loss=54.3656
mps
[3] layers=2, neurons=[66, 80], lr=1.1e-03, val_loss=153.8862
mps
[4] layers=3, neurons=[19, 125, 42], lr=8.5e-03, val_loss=18.9664
mps
[5] layers=4, neurons=[68, 29, 140, 94], lr=9.8e-04, val_loss=36.0448
mps
[6] layers=1, neurons=[183], lr=1.2e-03, val_loss=1431.0320
mps
[7] layers=2, neurons=[194, 157], lr=7.0e-03, val_loss=17.4525
mps
[8] layers=1, neurons=[47], lr=4.5e-03, val_loss=80.0933
mps
[9] layers=2, neurons=[63, 113], lr=9.4e-03, val_loss=21.3881
mps
[1] layers=3, neurons=[19, 125, 42], lr=8.5e-03, val_loss=20.4591
mps
[2] layers=4, neurons=[19, 125, 37, 162], lr=9.4e-03, val_loss=18.8059
mps
[3] layers=3, neurons=[63, 113, 37], lr=9.8e-04, val_loss=34.8907
mps
[4] layers=4, neurons=[68, 68, 110, 92], lr=1.7e-03, val_loss=23.8914
mps
[5] layers=2, neurons=[45, 15

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(fitness_per_gen, marker='s', linestyle='--', color='darkorange', label='Best Per Generation')
plt.title("Best Validation Loss Per Generation")
plt.xlabel("Generation")
plt.ylabel("Best Validation Loss (MSE)")
plt.grid(True)
plt.tight_layout()
plt.legend()
plt.show()


In [ ]:
fitness_over_time = []
def objective(params):
    try:
        n_hidden = int(round(params[0]))
        neurons = [int(round(p)) for p in params[1:6]]
        log_lr = params[6]
        learning_rate = 10 ** log_lr

        model = MLP(input_dim=X_train.shape[1], hidden_sizes=neurons[:n_hidden])
        model.to(device)

        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        X_torch = torch.tensor(X_train, dtype=torch.float32).to(device)
        y_torch = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)

        val_split = 0.2
        val_size = int(len(X_torch) * val_split)
        train_X, val_X = X_torch[:-val_size], X_torch[-val_size:]
        train_y, val_y = y_torch[:-val_size], y_torch[-val_size:]

        batch_size = 64
        train_dataset = torch.utils.data.TensorDataset(train_X, train_y)
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        best_val_loss = float('inf')
        epochs_no_improve = 0
        patience = 5

        for epoch in range(10):  # match TF version
            model.train()
            for xb, yb in train_loader:
                optimizer.zero_grad()
                preds = model(xb)
                loss = criterion(preds, yb)
                loss.backward()
                optimizer.step()

            model.eval()
            with torch.no_grad():
                val_preds = model(val_X)
                val_loss = criterion(val_preds, val_y).item()

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    break

        print(f"DE: layers={n_hidden}, neurons={neurons[:n_hidden]}, lr={learning_rate:.1e}, val_loss={best_val_loss:.4f}")
        return best_val_loss

    except Exception as e:
        print(f"DE FAILED: {e}")
        return 1e6



In [ ]:
bounds = [
    (1, 5),          # n_hidden
    (10, 200),       # n1
    (10, 200),       # n2
    (10, 200),       # n3
    (10, 200),       # n4
    (10, 200),       # n5
    (-4, -2)         # log10(lr)
]

# Differential Evolution with GA seed
result = differential_evolution(
    objective,
    bounds,
    strategy="best1bin",
    maxiter=2,
    popsize=5,
    seed=42,
    x0=best_solution,
    disp=True,
    polish=False
)

best_params = result.x
print("Refined DE solution:", best_params)


In [ ]:

plt.figure(figsize=(8, 5))
plt.plot(fitness_over_time, marker='o', linestyle='-', label='Validation Loss')
plt.title("Validation Loss Over DE Evaluations")
plt.xlabel("Evaluation Step")
plt.ylabel("Validation Loss (MSE)")
plt.grid(True)
plt.tight_layout()
plt.legend()
plt.show()


In [ ]:
# Unpack final parameters
n_hidden_best = int(round(best_params[0]))
neurons_best = [int(round(p)) for p in best_params[1:6]]
log_lr_best = best_params[6]
learning_rate_best = 10 ** log_lr_best

print(f"Final architecture: layers={n_hidden_best}, neurons={neurons_best[:n_hidden_best]}, lr={learning_rate_best:.1e}")

# Final model training
final_model = create_mlp(n_hidden_best, neurons_best, learning_rate_best)
final_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=64,
    verbose=1,
    callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
)

# Evaluate on test set
mse_test = final_model.evaluate(X_test, y_test, verbose=0)
rmse_test = np.sqrt(mse_test)
print(f"Final test MSE: {mse_test:.4f}, RMSE: {rmse_test:.2f} MW")

from sklearn.metrics import r2_score

y_pred = final_model.predict(X_test).flatten()
r2 = r2_score(y_test, y_pred)
print(f"R² score: {r2:.4f}")



In [ ]:
final_model.save("ccpp_best_model.h5")

from sklearn.metrics import r2_score

# Predictions and R²
y_pred = final_model.predict(X_test).flatten()
rmse = np.sqrt(np.mean((y_test - y_pred)**2))
r2 = r2_score(y_test, y_pred)

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5, s=20, c="dodgerblue", edgecolors="k", linewidths=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=1.5)

# Labels and title
plt.title(f"CCPP: Actual vs Predicted Power Output\nRMSE = {rmse:.2f} MW | R² = {r2:.4f}", fontsize=13)
plt.xlabel("Actual PE (MW)", fontsize=12)
plt.ylabel("Predicted PE (MW)", fontsize=12)

# Ticks and limits
plt.xlim(425, 500)
plt.ylim(425, 500)
plt.xticks(np.arange(430, 501, 10))
plt.yticks(np.arange(430, 501, 10))

# Grid and layout
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

